# Capacitor sourcing workbook

Use this workflow to build a robust standard capacitor library that is resilient to marketplace fluctuation.

In [163]:
%load_ext autoreload
import sys
import octopart
sys.path.append("../")
import scottopart.constants
import scottopart.mpnmagic.capacitors

octopart.logger.setLevel(20)  # disable annoying debug logs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [164]:
authorized_capacitor_manufacturers = ['TDK', 'Murata', 'Yageo', 'KEMET', 'Taiyo Yuden', 'Samsung', 'AVX']
authorized_sellers = ['Digi-Key', 'Newark', 'Mouser', 'Avnet', 'Arrow Electronics, Inc.']
authorized_country_codes = ['US']
authorized_packaging = ['Cut Tape', 'Tape & Reel', 'None']
query_package = ['0603']
query_capacitance = [4.7e-9]

for case_package in query_package:
    for capacitance in query_capacitance:
        print(case_package, capacitance)

0603 4.7e-09


In [165]:
# We would do this if we can, but the octopart Python API
# won't accept a list for manufacturer.displayname.  So this
# is a work-around.
#filter = {
#    'manufacturer.displayname' : ['Murata', 'TDK', 'etc.']
#    'specs.case_package.value' : '0603',
#    'specs.capacitance.value'  : '4.7e-9'
#}
#r = octopart.search(query='', limit=100, filter_fields=filter, include_specs=True)

c = octopart.client.OctopartClient()
params = {
        'q': '', # leave blank
        'filter[fields][manufacturer.displayname][]': authorized_capacitor_manufacturers,
        'filter[fields][specs.capacitance.value][]': capacitance,
        'filter[fields][specs.case_package.value][]': case_package,
        'limit' : 100,
        'include[]' : 'specs',
}
r = octopart.models.PartsSearchResult(c._request('/parts/search', params=params))

In [166]:
r

<PartsSearchResult: hits=522>

In [230]:
# TODO: pick up authorized_* from a more sensible location
# TODO: figure out Digi-Key double-counting of in-stock quantity
def authorized_in_stock_qty(part):
    """Useful as a sorting key for a list of Part objects."""
    filtered_offers = [o for o in part.offers if o.seller in authorized_sellers and o.packaging in authorized_packaging]
    return sum([o.in_stock_quantity for o in filtered_offers])

In [222]:
sorted_parts = sorted(r.parts, key=authorized_in_stock_qty, reverse=True)

In [242]:
for p in sorted_parts:
    print(p, authorized_in_stock_qty(p), scottopart.mpnmagic.capacitors.parse_mpn(p.mpn))

<Part mpn=GRM033R61A472KA01D> 330000 {'manufacturer_series': 'GRM', 'case_package': '0201', 'size_thickness': '0.3 mm', 'dielectric_characteristic': 'X5R', 'voltage_rating_dc': 10, 'capacitance': 4.7e-09, 'capacitance_tolerance': '10%', 'spec_code': 'A01', 'packaging_code': 'D', 'manufacturer': 'Murata'}
<Part mpn=GRM033R71A472KA01D> 315000 {'manufacturer_series': 'GRM', 'case_package': '0201', 'size_thickness': '0.3 mm', 'dielectric_characteristic': 'X7R', 'voltage_rating_dc': 10, 'capacitance': 4.7e-09, 'capacitance_tolerance': '10%', 'spec_code': 'A01', 'packaging_code': 'D', 'manufacturer': 'Murata'}
<Part mpn=C0603C472J5RACTU> 172595 {'case_package': '0603', 'manufacturer_series': 'C', 'capacitance': 4.7e-09, 'capacitance_tolerance': '5%', 'voltage_rating_dc': 50, 'dielectric_characteristic': 'X7R', 'reliability_code': 'A', 'plating_code': 'C', 'packaging_code': 'TU', 'manufacturer': 'Kemet'}
<Part mpn=GCM188R71H472KA37J> 163756 {'manufacturer_series': 'GCM', 'case_package': '0603

In [169]:
p = r.parts[0] # for testing

In [172]:
p.offers[0].prices

{'USD': {1: 0.1, 10: 0.04, 100: 0.0176, 500: 0.0126, 1000: 0.0099}}

In [116]:
parsed_parts[10]

{'case_package': '0603',
 'voltage_rating_dc': 100,
 'dielectric_characteristic': 'X7R',
 'capacitance': 4.7e-09,
 'capacitance_tolerance': '10%',
 'reliability_code': 'A',
 'plating_code': 'Z',
 'packaging_code': '2',
 'special_code': 'A',
 'manufacturer': 'AVX'}

In [158]:
# filter the part offers
filtered_offers = [o for o in p.offers if o.seller in authorized_sellers and o.packaging in authorized_packaging]

In [160]:
authorized_in_stock_qty = sum([o.in_stock_quantity for o in filtered_offers])

In [161]:
authorized_in_stock_qty

106216

In [152]:
filtered_o[2].packaging

'Cut Tape'

In [173]:
import pandas as pd

In [174]:
df = pd.DataFrame()

In [181]:
df = df.append({'Manufacturer': "TDK", 'Key2' : 'val3'}, ignore_index=True)

In [184]:
df

,Key2,Manufacturer
0,val2,Murata
1,val3,TDK


In [196]:
sorted(r.parts, key = lambda x: x.mpn)

[<Part mpn=06031C472JAT2A>,
 <Part mpn=06031C472K4T2A>,
 <Part mpn=06031C472K4Z2A>,
 <Part mpn=06031C472KAT2A>,
 <Part mpn=06031C472KAZ2A>,
 <Part mpn=06031C472MAT2A>,
 <Part mpn=06033C472JAT2A>,
 <Part mpn=06033C472KAT2A>,
 <Part mpn=06033C472MAT2A>,
 <Part mpn=06035C472JAT2A>,
 <Part mpn=06035C472JAT4A>,
 <Part mpn=06035C472K4T2A>,
 <Part mpn=06035C472K4T4A>,
 <Part mpn=06035C472K4Z2A>,
 <Part mpn=06035C472KAT2A>,
 <Part mpn=06035C472KAT4A>,
 <Part mpn=06035C472KAZ2A>,
 <Part mpn=06035C472MAT2A>,
 <Part mpn=06035F472K4T2A>,
 <Part mpn=0603YC472JAT2A>,
 <Part mpn=0603YC472KAT2A>,
 <Part mpn=0603ZC472MAT2A>,
 <Part mpn=C0603C472F3GACTU>,
 <Part mpn=C0603C472F5GACTU>,
 <Part mpn=C0603C472G5GACTU>,
 <Part mpn=C0603C472J1GACAUTO>,
 <Part mpn=C0603C472J1GACTU>,
 <Part mpn=C0603C472J1HACTU>,
 <Part mpn=C0603C472J1RACTU>,
 <Part mpn=C0603C472J3GACTU>,
 <Part mpn=C0603C472J5GACAUTO>,
 <Part mpn=C0603C472J5GACTU>,
 <Part mpn=C0603C472J5RACAUTO>,
 <Part mpn=C0603C472J5RACTU>,
 <Part mpn=C0603C4

In [214]:
authorized_in_stock_qty(r.parts[1])

172595

In [218]:
r.parts

[<Part mpn=C0603C472K5RACTU>,
 <Part mpn=C0603C472J5RACTU>,
 <Part mpn=C0603C472J5GACTU>,
 <Part mpn=C0603C472K1RACTU>,
 <Part mpn=C0603C472K2RACTU>,
 <Part mpn=06031C472KAT2A>,
 <Part mpn=C0603C472F5GACTU>,
 <Part mpn=C0603C472J1GACTU>,
 <Part mpn=06035C472KAT2A>,
 <Part mpn=C1608C0G1H472J080AA>,
 <Part mpn=06031C472KAZ2A>,
 <Part mpn=06035C472JAT2A>,
 <Part mpn=C0603C472K4RACTU>,
 <Part mpn=C0603C472K3RACTU>,
 <Part mpn=06031C472JAT2A>,
 <Part mpn=C0603C472J3GACTU>,
 <Part mpn=C0603C472K5RACAUTO>,
 <Part mpn=GRM1885C1H472JA01D>,
 <Part mpn=06031C472K4T2A>,
 <Part mpn=C0603X472K5RACTU>,
 <Part mpn=CL10B472KB8NNNC>,
 <Part mpn=CC0603KRX7R9BB472>,
 <Part mpn=CGA3E2X7R1H472K080AA>,
 <Part mpn=GRM033R71A472KA01D>,
 <Part mpn=C0603C472K2RACAUTO>,
 <Part mpn=ESD35C472K4T2A-18>,
 <Part mpn=06035F472K4T2A>,
 <Part mpn=CGA3E2C0G1H472J080AA>,
 <Part mpn=06035C472KAZ2A>,
 <Part mpn=C0603C472K8RACTU>,
 <Part mpn=C0603C472F3GACTU>,
 <Part mpn=C0603C472K1RACAUTO>,
 <Part mpn=GRM188R72A472KA01D>,
 <

In [226]:
p.__class__

octopart.models.Part

In [240]:
scottopart.mpnmagic.capacitors.parse_mpn('C0603H472J1GACTU')

{'case_package': '0603',
 'manufacturer_series': 'H',
 'capacitance': 4.7e-09,
 'capacitance_tolerance': '5%',
 'voltage_rating_dc': 100,
 'dielectric_characteristic': 'C0G/NP0',
 'reliability_code': 'A',
 'plating_code': 'C',
 'packaging_code': 'TU',
 'manufacturer': 'Kemet'}